<a href="https://colab.research.google.com/github/aadarsjb/recommendation-system/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing pandas and numpy

In [65]:
import pandas as pd
import numpy as np

importing movies credit datasets

In [66]:
credits = pd.read_csv("tmdb_5000_credits.csv")

importing movies datasets

In [67]:
movies_df = pd.read_csv("tmdb_5000_movies.csv")

viewing the datasets of credit

In [ ]:
credits.head()

viewing the datasets of movies

In [ ]:
movies_df.head()

In [70]:
print("Credits:", credits_df.shape)
print("Movies Dataframe:", movies_df.shape) 

Credits: (4803, 4)
Movies Dataframe: (4803, 20)


In [71]:
credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

renaming the credits column movie_id with id

In [52]:
credits_column_renamed=credits.rename(columns={'movie_id': 'id'}, inplace=True)


In [72]:
display('credits', 'movies_df', "pd.merge(credits, movies_df, on='id')")

'credits'

'movies_df'

"pd.merge(credits, movies_df, on='id')"

merging datasets

displaying merged datasets

In [ ]:
movies_df_merge.head(2)

dropping unnecessary columns:

In [75]:
movies_cleaned_df = movies_df_merge.drop(columns=['homepage', 'title_x', 'title_y', 'status', 'production_countries'])

In [ ]:
movies_cleaned_df.head(2)

Content Based Recommendation System

In [77]:
movies_cleaned_df.head(1)['overview']

0    In the 22nd century, a paraplegic Marine is di...
Name: overview, dtype: object

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [82]:
tfv = TfidfVectorizer(
                      min_df = 3, max_features = None, 
                      strip_accents = 'unicode',
                      analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range= (1,3),
                      stop_words= 'english' 
                      )


In [85]:
#filling NaNs with empty string
movies_cleaned_df['overview'] = movies_cleaned_df['overview'].fillna('')

In [86]:
#fitting the TF-IDF on the'overview' text
tfv_matrix = tfv.fit_transform(movies_cleaned_df['overview'])

In [87]:
tfv_matrix

<4803x10417 sparse matrix of type '<class 'numpy.float64'>'
	with 127220 stored elements in Compressed Sparse Row format>

In [88]:
tfv_matrix.shape

(4803, 10417)

In [94]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [95]:
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [96]:
sig[0]

array([0.76163447, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [97]:
#reverse mapping of indices and movie titles
indices = pd.Series(movies_cleaned_df.index, index=movies_cleaned_df['original_title']).drop_duplicates()

In [ ]:
indices

In [99]:
indices['Newlyweds']

4799

In [100]:
sig[4799]

array([0.76159416, 0.76159416, 0.76159438, ..., 0.76159432, 0.76159416,
       0.76159478])

In [ ]:
list(enumerate(sig[indices['Newlyweds']]))

In [ ]:
sorted(list(enumerate(sig[indices['Newlyweds']])), key = lambda x: x[1], reverse = True)

In [104]:
def give_rec(title, sig = sig):
  #Get the index corresponding to original_title
   idx= indices[title]

   #get the pairwise similarity scores
   sig_scores = list(enumerate(sig[idx]))

   #sort the movies
   sig_scores= sorted(sig_scores, key=lambda x: x[1], reverse=True)

   #scores of the 10 most similar movies
   sig_scores = sig_scores[1:11]

   #movie indices
   movie_indices = [i[0] for i in sig_scores]

   #top 10 most similar movies
   return movies_cleaned_df['original_title'].iloc[movie_indices] 

In [107]:
#testing ouir system
give_rec('Avatar')

1341                Obitaemyy Ostrov
634                       The Matrix
3604                       Apollo 18
2130                    The American
775                        Supernova
529                 Tears of the Sun
151                          Beowulf
311     The Adventures of Pluto Nash
847                         Semi-Pro
942                 The Book of Life
Name: original_title, dtype: object